## 1) Se cuenta con un RDD con información sobre patentamientos de autos con la siguiente información 
#### (patente, marca, modelo,versión, tipo_vehiculo, provincia, fecha) 
##donde tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto.
##Se pide generar un programa en pySpark que indique la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2017. 

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext
PatentamientosRDD = sc.parallelize(Patentamientos, 8)

In [ ]:
#Filtro por provincia de Buenos Aires
Patentamientos = PatentamientosRDD.filter(lambda x: x[5] == 'Buenos Aires')

In [ ]:
#Filtro por Mes y Año
Patentamientos = Patentamientos.filter(lambda x: x[6].startswith('2017-04'))

In [ ]:
#Reorganizo el RDD para poder realizar el reduceByKey ( (tipo_vehiculo, marca, modelo), 1  )
Patentamientos = Patentamientos.map(lambda x: ((x[4], x[1], x[2]),1)

In [ ]:
#Sumo las cantidades de patentamientos ( (tipo_vehiculo, marca, modelo), cant_totales_patentamientos  )
Patentamientos = Patentamientos.reduceByKey(lambda a,b: a+b)

In [ ]:
#Reorganizo mi RDD de la forma ( tipo_vehiculo, ( (marca, modelo) , cant_totales_patentamientos ) ) 
Patentamientos = Patentamientos.map(lambda x: ( x[0][0],( (x[0][1], x[0][2]) , x[1] ) ) 

In [ ]:
#Me quedo con el mas patentado. 
LosMasPatentados = Patentamientos.reduceByKey(lambda a,b: a if a[1] > b[1] else b )

In [ ]:
#Resultado Final --> (tipo_vehiculo, marca, modelo)
LosMasPatentados = LosMasPatentados.map(lambda x: (x[0], x[1][0][0], x[1][0][1]))